///---
layout: post
title: "CUDA"
comments: true
share: true
date: 2020-05-30 01:00:00
description: Cuda에 대해 정리한다.
tags: cuda
toc: true
sitemap :
  changefreq : daily
  priority : 1.0
///---

# CUDA

## Memory

### Memory hierarchy

<figure>
    <img src='../assets/images/Cuda/memory_hierarchy.png' alt='Memory hierarchy' width="640" />
    <figcaption class="figure-caption">Memory hierarchy</figcaption>
</figure>

- Registers
  - Thread 간에도 공유되지 않는다.
  - 함수에서 선언한 변수를 저장한다.
  - 함수에서 선언한 변수의 크기가 Thread의 Register개수를 넘을 경우 Local 메모리에 위치한다.
- Shared memory
  - __shared__를 이용하여 선언한다.
  - Local, Global memory보다 지연시간이 적다.
  - Block내의 Thread간 공유된다.
  - __syncThreads()으로 Shared memory의 Coherent를 유지한다.
  - L1 cache와 하드웨어를 공유하고 할당량은 cudaFuncSetCache함수를 이용하여 설정가능한다.
- Local memory
  - Registers의 공간 부족으로 Cache에 위치한 변수
- Constant memory
  - __constant__를 이용하여 선언한다.
  - Global scope 선언되어야 한다.
  - Kernel 실행 전 cudaMemcpyToSymbo로 값을 설정하여야한다.
- Texture memory
  - Read Only
  - 2D Spatial Locality에 최적회 되어 있다.
- Global memory
  - __device__ 식별자를 이용하여 선언하거나 Host에서 cudaMalloc을 이용한다.

  
  
---

- CUDA Variable and Type Qualifier

|   Qualifier  |  Variable Name |  Memory  |  Scope |   Lifespan  |
|:------------:|:--------------:|:--------:|:------:|:-----------:|
|              |    float var   | Register | Thread |    Thread   |
|              | float var[100] |   Local  | Thread |    Thread   |
|  __shared__  |    float var   |  Shared  |  Block |    Block    |
|  __device__  |    float var   |  Global  | Global | Application |
| __constant__ |    float var   | Constant | Global | Application |

- Salient Features of Device Memory

|  Memory  | On/Off Chip | Cached  | Access |        Scope         |     Lifetime    |
|:--------:|:-----------:|:-------:|:------:|:--------------------:|:---------------:|
| Register |      On     |   n/a   |   R/W  |       1 thread       |      Thread     |
|   Local  |     Off     |   Yes   |   R/W  |       1 thread       |      Thread     |
|  Shared  |      On     |   n/a   |   R/W  | All threads in block |      Block      |
|  Global  |     Off     |    †    |   R/W  |  All threads + host  | Host allocation |
| Constant |     Off     |   Yes   |    R   |  All threads + host  | Host allocation |
|  Texture |     Off     |   Yes   |    R   |  All threads + host  | Host allocation |

### Host <-> Device
Host와 device의 변수가 동일한 파일에 선언되어 있어도 직접적인 참조는 불가능한다. 배열의 경우 cudaAlloc으로 할당된 포인터와 cudaMemcpy를 이용하여 Host와 device간 데이터를 교환할 수 있다. 하지만 Global scope에 __device__로 선언된 변수의 경우는 변수의 주소값과 cudaMemcpy를 이용하여 값을 전송 할 수 없다. 이런 경우에는 cudaGetSymboAddress를 이용하여 포인터를 얻어와서 cudaMemcpy를 이용하거나 cudaMemcpyToSymbol 명령을 이용하여 한다.

```c
#include <cuda_runtime.h>
#include <stdio.h>
__device__ float devData;
__global__ void checkGlobalVariable() {
   devData +=2.0f;
}
int main(void) {
   float value = 3.14f;
   cudaMemcpyToSymbol(devData, &value, sizeof(float));
   // 위의 처럼 devData에 저장하거나
   float *dptr = NULL;
   cudaGetSymbolAddress((void**)&dptr, devData);
   cudaMemcpy(dptr, &value, sizeof(float), cudaMemcpyHostToDevice);
   //
   checkGlobalVariable <<<1, 1>>>();
   cudaMemcpyFromSymbol(&value, devData, sizeof(float));
   cudaDeviceReset();
   return EXIT_SUCCESS;
}
```

### Pinned Memory

CPU에서 선언한 배열의 경우 Pageable 메모리 형태로 생성되므로 Device로 데이터를 전송 할때 Overhead가 생성된다. Host에서 사용할 메모리를 cudaMallocHost/cudaFreeHost를 이용하면 Pinned Memory(Non-pageable)로 선언되므로 Overhead를 줄일 수 있다. Pinned Memory가 많을 수록 host의 전체 성능이 저하 될 수 있으므로 시스템과 프로그램 상황에 맞게 조절되어야 한다.


<figure>
    <img src='../assets/images/Cuda/pinned_memroy.png' alt='Pinned Memory' width="640" />
    <figcaption class="figure-caption">Pinned Memory</figcaption>
</figure>

### Zero-Copy Memory

일반적으로는 Host<->Device간의 데이터를 직접 주고 받을 수 없지만 Zero-Copy memory로 선언된 영역에 대해서는 Host와 device가 접근할 수 있다. cudaHostAlloc/cudaFreeHost를 이용하여 선언/해제 할 수 있다.

Zero-Copy Memory의 장점은 다음과 같다.

- Leveraging host memory when there is insufficient device memory
- Avoiding explicit data transfer between the host and device
- Improving PCIe transfer rates

큰 배열의 경우 cudaMalloc을 사용하는 것이 훨씬 효율적이다.

## 참고문헌
[1] Professional CUDA C Programming 